In [2]:
from  scipy.special import j1
from kSZ_forecast_general_func import *
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
Grid = 512
MAS  = "NGP"
Snapshot = 2448

snap_info=Snap_Info()

Thelen = Boxlen/h/cosmo.comoving_distance(z=snap_info.get_redshift(Snapshot)).value
cnorm=ne0*sigma_t*Mpc2m*Boxlen*1.0/Grid*(1+snap_info.get_redshift(Snapshot))**2/h  # proj mom: km/h -> dkSZ/CMB

the_los=(45)*np.pi/180
n_rsd=np.array([np.cos(the_los),np.sin(the_los),0])

if Snapshot == 1631:
    n_gal = np.array([3, 3.3, 3.6])*10**-4
if Snapshot == 2448:
    n_gal = np.array([6, 6.6, 7.2])*10**-4
n_halo =np.array([    1,     2,     3,     5,    7,     9,    30,   70])*10**-4 #8
lgM_min=np.array([12.86, 12.68, 12.55, 12.41, 12.3, 12.23, 11.82, 11.5])

In [7]:
4*np.pi*(180/np.pi)**2

41252.96124941928

In [8]:
def cal_survey_volume(z1,z2,fs):
    h=0.69
    d1=cosmo.comoving_distance(z=z1).value*h
    d2=cosmo.comoving_distance(z=z2).value*h
    V = 4*np.pi/3*(d2**3-d1**3)
    f = fs/(4*np.pi*(180/np.pi)**2)
    return f*V/10**9
cal_survey_volume(z1=0.6, z2=1, fs=1400)

1.210845584131293

In [2]:
Den_dir= "/home/chenzy/data/denmap/den"+MAS+"_"+str(Grid)+"_"+str(Snapshot)+"_0"
Mom_dir = "/home/chenzy/data/mommap/mom"+MAS+"_"+str(Grid)+"_"+str(Snapshot)+"_0"
momp=snap_info.get_vel_norm(Snapshot)*np.fromfile(Mom_dir, dtype=np.float32, count=3*Grid**3, sep="").reshape(Grid,Grid,Grid,3)
denp=np.fromfile(Den_dir, dtype=np.float32, count=Grid**3, sep="").reshape((Grid,)*3)
velp = momden2vel(denp, momp)
momp = (denp-1).reshape(Grid, Grid, Grid,1)*velp
momp_proj=proj1(momp,the_los)*cnorm

In [5]:
cmb_map_act = cmb2d(thelen=Thelen, grid = Grid, Delta_T=cmbsur[0][0],sigma=cmbsur[0][1])

In [23]:
print(Snapshot, Thelen)
for i in range(100):
    filename = "./CMB_maps/ACT_"+str(Snapshot)+"_"+str(i).zfill(3)+".npy"
    cmb_map_act = cmb2d(thelen=Thelen, grid = Grid, Delta_T=cmbsur[0][0],sigma=cmbsur[0][1])
    np.save(filename, cmb_map_act)

In [3]:
halox,halov, mh =ReadHalos_np(zstep=Snapshot)
halo_mass = mh*pm

# stack

In [8]:
galaxy_number_density_label = 2
#mass_selection_label2 = 0
den_method = 0   #NGP
vel_method = 0   #thetap
HOD_model = "HSC_NB912"

N_jk = 100
the_ap = np.linspace(1, 6, 10)
haloxx_rsd=add_RSD(halox*Boxlen,halov*(1+snap_info.get_redshift(Snapshot)), n_rsd, cosmo.H(snap_info.get_redshift(Snapshot)).value)%Boxlen   #Mpc/h
    
#halo sample for velocity reconstruction(for HOD_method)
richness = np.load("./halo_Temporary_storage/halos_"+str(Snapshot)+"_richness_"+HOD_model+".npy")
N_gal = np.sum(richness)
print("Number of galaxies", N_gal,"   n=",N_gal/Boxlen**3)
galaxy_pos = np.zeros((N_gal, 3))
for i in range(3):
    galaxy_pos[:,i]=np.repeat(haloxx_rsd[:,i], richness)

    
#rand to make density (for n_gal)
N_gal_need = np.int32(n_gal[galaxy_number_density_label]*Boxlen**3)
a = np.arange(N_gal, dtype=np.int32)
np.random.shuffle(a)
galaxy_pos = galaxy_pos[list(a[:N_gal_need]),:]
N_gal = len(galaxy_pos[:,0])
print("Number of galaxies", N_gal,"   n=",N_gal/Boxlen**3)

# for den_method
if den_method == 0:
    deng_rsd=den_NGP(galaxy_pos,Grid)
    
#for vel_method
if vel_method == 0:
    thetap = vel2theta(velp, Boxlen)
    W,kx,ky,kz=wiener_filter_theta_esti(deng_rsd,thetap, Boxlen)
    #del thetap
    thetag_rsd_rec=wiener_filter_den2theta(deng_rsd,W,kx,ky,kz, Boxlen)
    #del W,kx,ky,kz
    velg_rsd_rec=theta2vel(thetag_rsd_rec, Boxlen)

for j in range(5,8):
    mass_selection_label2 = j
    lgM_min2 = lgM_min[mass_selection_label2]
    label2 = np.where(halo_mass>10**lgM_min2)[0]
    n_halo2 = n_halo[mass_selection_label2]
    print("Halo sample to stack: n = "+str(n_halo2*10**4)+"$10^{-4}$ lgM_min "+str(lgM_min2))
    #'''
    for k in tqdm(range(30, 50)):
        filename = "/home/chenzy/code/kSZ_forecast/stack_prediction_results/S"+str(Snapshot)+"G"+str(Grid)+"_gal_"+HOD_model+"_den"+str(den_method)+"vel"+str(vel_method)+"l"+str(galaxy_number_density_label)+"l"+str(mass_selection_label2)+"ACT"+str(k).zfill(3)
        print(filename)
        cmb_map_act = np.load("./CMB_maps/ACT_"+str(Snapshot)+"_"+str(k).zfill(3)+".npy")
        stack_signal, r_true_rec, r_ksz_rec = cal_stack_kSZ_signal_proj1_jk(the_ap = the_ap, CMB_map = momp_proj+cmb_map_act, vel_rec=velg_rsd_rec, halox=halox[label2,:], halov=halov[label2, :], the_los=the_los, N_jk=N_jk, thelen=Thelen, grid = Grid)
        stack_mean, corv = error_esti_jackknife(stack_signal)
        C_ = pseudo_inverse(corv,2)
        chi_null = cal_chi_square(stack_mean, stack_mean, C_)

        print("S/N=",np.round(np.sqrt(chi_null), 2))
        np.savez(filename, stack_signal=stack_signal, r_true_rec =r_true_rec, r_ksz_rec=r_ksz_rec)
    break

KeyboardInterrupt: 

# tomography

In [6]:
Grid = 512
Snapshot = 2448
Thelen = Boxlen/h/cosmo.comoving_distance(z=snap_info.get_redshift(Snapshot)).value
cnorm=ne0*sigma_t*Mpc2m*Boxlen*1.0/Grid*(1+snap_info.get_redshift(Snapshot))**2/h  # proj mom: km/h -> dkSZ/CMB

if Snapshot == 1631:
    n_gal = np.array([3, 3.3, 3.6])*10**-4
if Snapshot == 2448:
    n_gal = np.array([6, 6.6, 7.2, 4, 4.4, 4.8])*10**-4

#galaxy_number_density_label = 2
den_method = 0   #NGP
vel_method = 0   #thetap
CMB_survey = "ACT"
#HOD_model = "HSC_NB816"
HOD_model = "DESI"

print("read DEN, MOM")
Den_dir= "/home/chenzy/data/denmap/den"+MAS+"_"+str(Grid)+"_"+str(Snapshot)+"_0"
Mom_dir = "/home/chenzy/data/mommap/mom"+MAS+"_"+str(Grid)+"_"+str(Snapshot)+"_0"
momp=snap_info.get_vel_norm(Snapshot)*np.fromfile(Mom_dir, dtype=np.float32, count=3*Grid**3, sep="").reshape(Grid,Grid,Grid,3)
denp=np.fromfile(Den_dir, dtype=np.float32, count=Grid**3, sep="").reshape((Grid,)*3)
velp = momden2vel(denp, momp)
momp = (denp-1).reshape(Grid, Grid, Grid,1)*velp
momp_proj=proj1(momp,the_los)*cnorm

print("read halo")
halox,halov, mh =ReadHalos_np(zstep=Snapshot)
halo_mass = mh*pm

haloxx_rsd=add_RSD(halox*Boxlen,halov*(1+snap_info.get_redshift(Snapshot)), n_rsd, cosmo.H(snap_info.get_redshift(Snapshot)).value)%Boxlen   #Mpc/h

pkden,kden=fa.CalPS(denp,Grid,mapscale=(Boxlen/Grid**2)**3,kscale=2*np.pi/Boxlen)
velp = momden2vel(denp, momp)
thetap = vel2theta(velp, Boxlen)

for i in [3,4,5]:
    galaxy_number_density_label = i
    #halo sample for velocity reconstruction(for HOD_method)
    if HOD_model == "DESI":
        richness = np.zeros(len(halo_mass), dtype = np.int32)
        for i in range(4):
            HOD_modeli = "DESI_L"+str(i)
            richness+=np.load("./halo_Temporary_storage/halos_"+str(Snapshot)+"_richness_"+HOD_modeli+".npy")
    else:
        richness = np.load("./halo_Temporary_storage/halos_"+str(Snapshot)+"_richness_"+HOD_model+".npy")
    N_gal = np.sum(richness)
    print("Number of galaxies", N_gal,"   n=",N_gal/Boxlen**3)
    galaxy_pos = np.zeros((N_gal, 3))
    for i in range(3):
        galaxy_pos[:,i]=np.repeat(haloxx_rsd[:,i], richness)

    #rand to make density (for n_gal)
    N_gal_need = np.int32(n_gal[galaxy_number_density_label]*Boxlen**3)
    a = np.arange(N_gal, dtype=np.int32)
    np.random.shuffle(a)
    galaxy_pos = galaxy_pos[list(a[:N_gal_need]),:]
    N_gal = len(galaxy_pos[:,0])
    print("Number of galaxies", N_gal,"   n=",N_gal/Boxlen**3)



    deng_rsd=den_NGP(galaxy_pos,Grid)
    W,kx,ky,kz=wiener_filter_theta_esti(deng_rsd,thetap, Boxlen)
    #del thetap
    thetag_rsd_rec=wiener_filter_den2theta(deng_rsd,W,kx,ky,kz, Boxlen)
    #del W,kx,ky,kz
    velg_rsd_rec=theta2vel(thetag_rsd_rec, Boxlen)


    deng_rsd_fw = wiener_filter_den(deng_rsd,pkden,kden,2*np.pi/Boxlen,40*10**-4,Grid)

    momg_rsd_rec=velg_rsd_rec*(deng_rsd_fw).reshape(Grid,Grid,Grid,1)
    momg_rsd_rec_proj = proj1(momg_rsd_rec, the_los)*cnorm

    cor, l = fa.CalCor2d(momg_rsd_rec_proj, momp_proj, Grid, mapscale=(Thelen/Grid**2)**2,kscale=2*np.pi/Thelen)
    ps_rec, l = fa.CalPS2d(momg_rsd_rec_proj, Grid, mapscale=(Thelen/Grid**2)**2,kscale=2*np.pi/Thelen)
    ps, l = fa.CalPS2d(momp_proj, Grid, mapscale=(Thelen/Grid**2)**2,kscale=2*np.pi/Thelen)

    CMB_survey = "ACT"
    l_array, NS, SN = SN_prediction_tomography(l=l, cor=cor, PS_rec=ps_rec, f_sky=1400/(4*np.pi*(180/np.pi)**2), CMB_survey=CMB_survey)
    print("S/N", SN)

    filename = "tomography_prediction_results/S"+str(Snapshot)+"G"+str(Grid)+"_gal_"+HOD_model+"_den"+str(den_method)+"vel"+str(vel_method)+"l"+str(galaxy_number_density_label)
    np.savez(filename, l=l, cor=cor, ps_rec=ps_rec, ps=ps, l_array=l_array, NS=NS, SN=SN)

read DEN, MOM
read halo
Number of galaxies 19300095    n= 0.011169036458333333
Number of galaxies 691200    n= 0.0004
S/N 4.628373059814749
Number of galaxies 19300095    n= 0.011169036458333333
Number of galaxies 760320    n= 0.00044
S/N 5.135167798622237
Number of galaxies 19300095    n= 0.011169036458333333
Number of galaxies 829440    n= 0.00048
S/N 5.4086076120117745


# figures

## stack S/N

In [ ]:
import os
n_halo =np.array([    1,     2,     3,     5,    7,     9,    30,   70])*10**-4 #8
lgM_min=np.array([12.86, 12.68, 12.55, 12.41, 12.3, 12.23, 11.82, 11.5])
den_method=0
vel_method=0
SN  = np.zeros((len(lgM_min), len(lgM_min)))

Thelen = Boxlen/h/cosmo.comoving_distance(z=snap_info.get_redshift(Snapshot)).value[0]
f_sky_factor = np.sqrt(1400/(Thelen*180/np.pi)**2)

for i in range(len(lgM_min)):
    for j in range(len(lgM_min)):
        filename = "stack_prediction_results/den"+str(den_method)+"vel"+str(vel_method)+"l"+str(i)+"l"+str(j)+".npz"
        if not os.path.exists(filename):
            continue
        d = np.load(filename)

        stack_signal = d["stack_signal"]
        stack_null = d["stack_null"]
        stack_mean, corv = error_esti_jackknife(stack_signal)
        C_ = pseudo_inverse(corv,2)
        chi_null = cal_chi_square(stack_mean, stack_mean, C_)
        SN[i, j] = np.sqrt(chi_null)

        
fig, ax = plt.subplots(figsize = (6,6))
im = ax.imshow(SN) 

label_nhalo = ["%.4f" % number for number in n_halo]
ax.set_xticks(np.arange(len(lgM_min)), labels=label_nhalo)
ax.set_yticks(np.arange(len(lgM_min)), labels=label_nhalo)

ax.set_ylabel("Halos for velocity reconstruction \n"+r"$n [\rm h/Mpc]^{3}$",fontsize=15)
ax.set_xlabel("Halos for stacking \n"+r"$n [\rm h/Mpc]^{3}$",fontsize=15)

plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(lgM_min)):
    for j in range(len(lgM_min)):
        text = ax.text(j, i, np.round(SN[i, j]*f_sky_factor, 2), ha="center", va="center", color="k")
        #print(SN[i, j], f_sky_factor, np.round(SN[i, j]*f_sky_factor, 1))

ax.set_title("S/N prediction: stacking method\n PFS "+r"$\times$ ACT")

plt.show()

## stack correlation parameter

In [ ]:
import os
n_halo =np.array([    1,     2,     3,     5,    7,     9,    30,   70])*10**-4 #8
lgM_min=np.array([12.86, 12.68, 12.55, 12.41, 12.3, 12.23, 11.82, 11.5])
den_method=0
vel_method=0
r  = np.zeros((len(lgM_min), len(lgM_min)))

Thelen = Boxlen/h/cosmo.comoving_distance(z=snap_info.get_redshift(Snapshot)).value[0]
f_sky_factor = np.sqrt(1400/(Thelen*180/np.pi)**2)

for i in [0,1,2,3,4,5,6,7]:
    for j in range(len(lgM_min)):
        filename = "stack_prediction_results/den"+str(den_method)+"vel"+str(vel_method)+"l"+str(i)+"l"+str(j)+".npz"
        if not os.path.exists(filename):
            continue
        d = np.load(filename)
        #print(filename)
        #r[i,j] = d["r_ksz_rec"][0]
        r[i,j] = d["r_true_rec"]


        
fig, ax = plt.subplots(figsize = (6,6))
im = ax.imshow(r, vmin=0.75) 
print(r)

label_nhalo = ["%.4f" % number for number in n_halo]
ax.set_xticks(np.arange(len(lgM_min)), labels=label_nhalo)
ax.set_yticks(np.arange(len(lgM_min)), labels=label_nhalo)

ax.set_ylabel("Halos for velocity reconstruction \n"+r"$n [\rm h/Mpc]^{3}$",fontsize=15)
ax.set_xlabel("Halos for stacking \n"+r"$n [\rm h/Mpc]^{3}$",fontsize=15)

plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(lgM_min)):
    for j in range(len(lgM_min)):
        text = ax.text(j, i, np.round(r[i,j], 4), ha="center", va="center", color="k")
        #print(SN[i, j], f_sky_factor, np.round(SN[i, j]*f_sky_factor, 1))

ax.set_title("Correlation parameter")

plt.show()

## tomography S/N

In [ ]:
SN = np.zeros(len(lgM_min))
for i in range(len(lgM_min)):
    filename = "tomography_prediction_results/den"+str(den_method)+"vel"+str(vel_method)+"l"+str(i)+".npz"
    if not os.path.exists(filename):
            continue
    d = np.load(filename)
    SN[i] = d["SN"]
    
fig, ax = plt.subplots()
plt.plot(n_halo, SN, '-ok')

plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
ax.set_xscale("log")
label_nhalo = ["%.4f" % number for number in n_halo]
ax.set_xticks(n_halo, labels=label_nhalo)
ax.set_xlabel("Halos for velocity reconstruction \n"+r"$n [\rm h/cMpc]^{3}$", fontsize=14)
ax.set_ylabel("S/N", fontsize=14)
ax.set_title("S/N prediction: Tomography method\n PFS "+r"$\times$ ACT", fontsize=14)
ax.grid()

In [1]:
5575630/1200**3

0.0032266377314814814